In [ ]:
#################################
###import neccesary libraries ###
#################################
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import os
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.client import device_lib
from pickle import load
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error
import random
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import shuffle
from mpl_toolkits.mplot3d import Axes3D
from keras import regularizers
from keras.utils import plot_model
from keras.layers.advanced_activations import PReLU, LeakyReLU
import math
from keras import optimizers
from keras import backend as K

In [ ]:
##############################
#######Storing Data   ########
##############################

np.random.seed(42)
random.seed(12345)
cwd = os.getcwd()
os.chdir(cwd)                                   #Working directory
filename = 'Puck_trained.sav'                   #File name
best_model = 'best_model.h5'                    #Early stopping for best model selection
def huber_loss(y_true, y_pred):
        return tf.losses.huber_loss(y_true,y_pred)
    
def mean_absolute_error_180(y_true, y_pred):
    delta = K.minimum(K.minimum(K.abs(y_pred - y_true),
                              K.abs(y_pred - (180+y_true))),
                              K.abs(y_true - (180+y_pred)))
    return K.mean(delta)

def absolute_error_180(y_true, y_pred):
    delta = K.minimum(K.minimum(K.abs(y_pred - y_true),
                              K.abs(y_pred - (180+y_true))),
                              K.abs(y_true - (180+y_pred)))
    return delta





def mean_sqared_error_180(y_true, y_pred):
    delta = K.minimum(K.minimum(K.abs(y_pred - y_true),
                              K.abs(y_pred - (180+y_true))),
                              K.abs(y_true - (180+y_pred)))
    return K.mean(K.square(delta))

def mean_sqared_squared_error_180(y_true, y_pred):
    delta = K.minimum(K.minimum(K.abs(y_pred - y_true),
                              K.abs(y_pred - (180+y_true))),
                              K.abs(y_true - (180+y_pred)))
    return K.mean(K.square(K.square(delta)))

def mean_absolute_error_eval(y_test, y_pred):
    error = np.empty([len(y_test), 1])
    for i in range(len(y_test)):
        error[i] = min(min(np.abs(y_pred[i] - y_test[i]),
                                  np.abs(y_pred[i] - (180+y_test[i]))),
                                  np.abs(y_test[i] - (180+y_pred[i])))
    return np.mean(error)



def rmse_360(y_true, y_pred):
    return K.sqrt(mean_squared_error_360(y_true, y_pred))

def atan2(x, y, epsilon=1.0e-12):
    x = tf.where(tf.equal(x, 0.0), x+epsilon, x)
    y = tf.where(tf.equal(y, 0.0), y+epsilon, y)    
    angle = tf.where(tf.greater(x,0.0), tf.atan(y/x), tf.zeros_like(x))
    angle = tf.where(tf.logical_and(tf.less(x,0.0),  tf.greater_equal(y,0.0)), tf.atan(y/x) + np.pi, angle)
    angle = tf.where(tf.logical_and(tf.less(x,0.0),  tf.less(y,0.0)), tf.atan(y/x) - np.pi, angle)
    angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.greater(y,0.0)), 0.5*np.pi * tf.ones_like(x), angle)
    angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.less(y,0.0)), -0.5*np.pi * tf.ones_like(x), angle)
    angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.equal(y,0.0)), tf.zeros_like(x), angle)
    return angle

def rmse_360_2(y_true, y_pred):
    return K.mean(K.abs(atan2(K.sin(y_true - y_pred), K.cos(y_true - y_pred))))


In [ ]:
##############################
#######  User inputs  ########
##############################

Sigma_min = -35                              #MPa
Sigma_max = 60                               #MPa
R_t_c = 240                                  #MPa
R_t_t = 67                                   #MPa
R_tp = 62                                    #MPa
R_tt = 47.4                                  #MPa
p_tt_c = 0.25
p_tt_t = 0.25
p_tp_t = 0.35
p_tp_c = 0.30
angle_steps = 180                            #number of angle steps (180 = 1° Schritte)
dataset_size = 10000                         #data size
Test_split_size = 0.3                        
Validation_split_size = 0.1                  #Wie viel Prozent des trainings Datasets sollten zum validieren verwendet werden
First_layer = 60                             #number of neurons in the layers
Secound_layer = 60                           
Third_layer = 60                             
Fourth_layer = 60                            
Fith_layer = 60                              
Sixth_layer = 60                             
loss_function = absolute_error_180           #User defined loss function
metric_function = 'mae'                      #metrics to measure loss
optimizer_function = 'adam'                  #Optimizer 
batch_size = 32                              #batch size
number_of_epochs = 500                       
kernel_initializer = 'he_uniform'
activation = 'linear'                          
early_stopping_monitoring = 'val_loss'   
early_stopping_patience = 10           
kernel_regularizer = regularizers.l1(0.01)    
activity_regularizer = None #regularizers.l1(0.01)                  

In [ ]:
##############################
########Daten generation########
##############################

#angle generation
winkel = np.linspace(0,180, angle_steps)

R_tt_A = R_t_c/(2*(1+p_tt_c))

#Stress generation
sigma_true = np.empty([5, dataset_size])

sigma_n = np.empty([len(winkel), dataset_size])
teta_nt = np.empty([len(winkel), dataset_size])
teta_nl = np.empty([len(winkel), dataset_size])

f_e = np.empty([len(winkel), dataset_size])

cos_psi = np.empty([len(winkel), dataset_size])
sin_psi = np.empty([len(winkel), dataset_size])

p_R_druck = np.empty([len(winkel), dataset_size])
p_R_zug = np.empty([len(winkel), dataset_size])

indizes = list()
failure_angle = np.empty([1, dataset_size])

for i in range(0, dataset_size):
    sigma_true[0, i] = random.uniform(Sigma_min, Sigma_max)
    sigma_true[1, i] = random.uniform(Sigma_min, Sigma_max)
    sigma_true[2, i] = random.uniform(Sigma_min, Sigma_max)
    sigma_true[3, i] = random.uniform(Sigma_min, Sigma_max)
    sigma_true[4, i] = random.uniform(Sigma_min, Sigma_max)


    

for i in range(dataset_size):
    for j in range(len(winkel)):
        sigma_n[j, i] =  sigma_true[0, i]*(math.cos(math.radians(winkel[j])))**2 + sigma_true[1, i]*(math.sin(math.radians(winkel[j])))**2 + 2* sigma_true[2, i]*math.sin(math.radians(winkel[j]))*math.cos(math.radians(winkel[j]))
        teta_nt[j, i] = -sigma_true[0, i]*math.sin(math.radians(winkel[j]))*math.cos(math.radians(winkel[j])) + sigma_true[1, i]*math.sin(math.radians(winkel[j]))*math.cos(math.radians(winkel[j])) + sigma_true[2, i]*((math.cos(math.radians(winkel[j])))**2 - (math.sin(math.radians(winkel[j])))**2)
        teta_nl[j, i] = sigma_true[3, i]*math.sin(math.radians(winkel[j])) + sigma_true[4, i]*math.cos(math.radians(winkel[j]))
        cos_psi[j, i] = teta_nt[j, i]**2/(teta_nt[j, i]**2 + teta_nl[j, i]**2)
        sin_psi[j, i] = teta_nl[j, i]**2/(teta_nt[j, i]**2 + teta_nl[j, i]**2)
        p_R_druck[j, i] = (p_tt_c/R_tt_A)*cos_psi[j, i] + (p_tp_c/R_tp)*sin_psi[j, i]
        p_R_zug[j, i] = (p_tt_t/R_tt_A)*cos_psi[j, i] + (p_tp_t/R_tp)*sin_psi[j, i]

        
for i in range(dataset_size):
    for j in range(len(winkel)):
        if sigma_n[j, i] >= 0:
            f_e[j, i] = math.sqrt(((1/R_t_t - p_R_zug[j, i])*sigma_n[j, i])**2 + (teta_nt[j, i]/R_tt_A)**2 + (teta_nl[j, i]/R_tp)**2) + p_R_zug[j, i]*sigma_n[j, i]
        else:
            f_e[j, i] = math.sqrt((teta_nt[j, i]/R_tt_A)**2 + (teta_nl[j, i]/R_tp)**2 + (p_R_druck[j, i]*sigma_n[j, i])**2) + p_R_druck[j, i] * sigma_n[j, i]
            
for i in range(dataset_size):
    index = np.argmax(np.max(f_e[:, i:i+1], axis=1))
    indizes.append(index)
    
for i in range(dataset_size):
    failure_angle[0, i] = winkel[indizes[i]]


X = sigma_true.T
y = failure_angle.T

In [ ]:
##############################
######Data preprocessing######
##############################

#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = Test_split_size, random_state = 0)
#Train validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = Validation_split_size, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_val = sc.transform(X_val)

In [ ]:
##############################
########Neuronal network#######
##############################

regressor = Sequential()

regressor.add(Dense(units = First_layer, input_dim = 5,
                    kernel_initializer = kernel_initializer, activation = activation,
                    kernel_regularizer = kernel_regularizer,
                    activity_regularizer = activity_regularizer))
#regressor.add(LeakyReLU(alpha=.1))
#regressor.add(Dropout(0.1))
regressor.add(Dense(units = Secound_layer, kernel_initializer = kernel_initializer,
                    activation = activation, kernel_regularizer = kernel_regularizer,
                    activity_regularizer = activity_regularizer))
#regressor.add(LeakyReLU(alpha=.1))
#regressor.add(Dropout(0.1))
regressor.add(Dense(units = Third_layer, kernel_initializer = kernel_initializer,
                    activation = activation, kernel_regularizer = kernel_regularizer,
                    activity_regularizer = activity_regularizer))
#regressor.add(LeakyReLU(alpha=.1))
#regressor.add(Dropout(0.1))
regressor.add(Dense(units = Fourth_layer, kernel_initializer = kernel_initializer,
                    activation = activation, kernel_regularizer = kernel_regularizer,
                    activity_regularizer = activity_regularizer))
#regressor.add(LeakyReLU(alpha=.1))
#regressor.add(Dropout(0.1))
regressor.add(Dense(units = Fith_layer, kernel_initializer = kernel_initializer,
                    activation = activation, kernel_regularizer = kernel_regularizer,
                    activity_regularizer = activity_regularizer))
#regressor.add(Dropout(0.1))
regressor.add(Dense(units = Sixth_layer, kernel_initializer = kernel_initializer,
                    activation = activation, kernel_regularizer = kernel_regularizer,
                    activity_regularizer = activity_regularizer))
#regressor.add(Dropout(0.1))
regressor.add(Dense(units = 1, kernel_initializer = kernel_initializer))

# Compiling the ANN
#optimizer_function = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
regressor.compile(optimizer = optimizer_function , loss = loss_function , metrics=[metric_function])

callbacks = [EarlyStopping(monitor = early_stopping_monitoring, patience = early_stopping_patience)]

# Check where the optimization is curried out CPU or GPU
print(device_lib.list_local_devices())
# Fitting the ANN to the training set
history = regressor.fit(X_train, y_train, 
                         batch_size = 32, epochs = 200, callbacks = callbacks,
                         validation_data=(X_val, y_val), verbose = 1)

# open saved best model
with open('trainHistoryOld', 'wb') as handle:
    pickle.dump(history.history, handle)
weights = regressor.get_weights()    
pickle.dump(regressor, open(filename, 'wb'))

In [ ]:
regressor = pickle.load(open(filename, 'rb'))

with open('trainHistoryOld', 'rb') as handle: 
    lernen = load(handle)

#Representation of error minimization over the epochs
plt.plot(lernen['mean_absolute_error'], color = 'red', label = 'Mean absolute error Trainingsdaten')
plt.plot(lernen['val_mean_absolute_error'], color = 'blue', label = 'Mean absolute error Validierungsdaten')
plt.title('model accuracy')
plt.legend()
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.show()

#Predict
y_pred = regressor.predict(X_test)
mean_absolute_error = metrics.mean_absolute_error(y_pred, y_test)
r2_failure_angle = metrics.r2_score(y_test, y_pred)
print('Der R²-Wert für den Versagenswinkel beträgt:',r2_failure_angle)
print('Der MAE für den Versagenswinkel beträgt:',mean_absolute_error)

In [ ]:
y_pred = regressor.predict(X_test)
mean_absolute_error = metrics.mean_absolute_error(y_pred, y_test)
r2_failure_angle = metrics.r2_score(y_test, y_pred)
print('Der R²-Wert für den Versagenswinkel beträgt:',r2_failure_angle)
print('Der MAE für den Versagenswinkel beträgt:',mean_absolute_error)

In [ ]:
#for i in range(len(y_pred)):
 #   if y_pred[i]<0:
  #      y_pred[i] = y_pred[i] + 180
plt.scatter(y_test, y_pred, s = 0.1, color = 'blue')
plt.title('Model')
plt.ylabel('predicted angle')
plt.xlabel('correct angle')
plt.show()
mean_absolute_error = mean_absolute_error_eval(y_pred, y_test)
error = np.empty([len(y_pred),1])
for i in range(len(y_pred)):
    error[i] = min(min(np.abs(y_pred[i] - y_test[i]),
                                  np.abs(y_pred[i] - (180+y_test[i]))),
                                  np.abs(y_test[i] - (180+y_pred[i])))
r2_failure_angle = metrics.r2_score(y_test, y_pred)


In [ ]:
kernel_initializer = ['he_uniform', 'lecun_uniform']
number_of_neurons = [60, 65, 70]
number_of_hidden_layers = [7, 8, 9, 10]
optimizer = ['rmsprop', 'adagrad', 'adam']
loss = [mean_absolute_error_180, mean_sqared_error_180]
param_list = grid_search(optimizer = optimizer, kernel_initializer = kernel_initializer, number_of_hidden_layers = number_of_hidden_layers, loss = loss, number_of_neurons = number_of_neurons)


In [ ]:
#######################
# Start of gred search#
#######################

columns = ['number of neurons', 'kernel initializer', 'optimizer', 'loss function',
           'batch sizes', 'number of hidden layers', 'MAE failure angle']
grid_search_list = pd.DataFrame(param_list, columns = columns)
grid_search_list = grid_search_list.sort_values(by='MAE failure angle')
grid_search_list

In [ ]:
def grid_search(**keyword_parameters):
    
    #Keywords abfragen und übernhemen/ Falls nciht vorhanden defaults setzen
    
    complete_time = time.time()
    if ('number_of_neurons' in keyword_parameters):
        number_of_neuron = number_of_neurons
    else:
        number_of_neuron = [4]
        
    if ('kernel_initializer' in keyword_parameters):
        kernel_initializers = kernel_initializer
    else:
        kernel_initializers = ['lecun_uniform']
        
    if ('optimizer' in keyword_parameters):
        optimizers = optimizer
    else:
        optimizers = ['adam']
        
    if ('loss' in keyword_parameters):
        losses = loss
    else:
        losses = ['logcosh']        
        
    if ('number_of_hidden_layers' in keyword_parameters):
        number_of_hidden_layer = number_of_hidden_layers
    else:
        number_of_hidden_layer = [1]

    if ('batch_size' in keyword_parameters):
        batch_sizes = batch_size
    else:
        batch_sizes = [32]
        
    
    number_of_runs = len(number_of_neuron)*len(kernel_initializers)*len(optimizers)*len(losses)*len(number_of_hidden_layer)*len(batch_sizes)
    param_list = np.empty([number_of_runs, 7], dtype = 'U25')
    run_counter = -1
    for a in range(len(number_of_neuron)):
        for b in range(len(kernel_initializers)):
            for c in range(len(optimizers)):
                for d in range(len(losses)):
                    for e in range(len(batch_sizes)):
                        hidden_layer_counter = number_of_hidden_layer[0] - 2
                        for f in range(len(number_of_hidden_layer)):
                            one_loop_time = time.time()
                            hidden_layer_counter = hidden_layer_counter + 1
                            run_counter = run_counter + 1
                            regressor = 0
                            regressor = Sequential()
                            regressor.add(Dense(units = number_of_neuron[a], input_dim = 5,
                                      kernel_initializer = kernel_initializers[b], activation = activation,
                                      kernel_regularizer = kernel_regularizer,
                                      activity_regularizer = activity_regularizer))
                            
                            for g in range(hidden_layer_counter):
                                regressor.add(Dense(units = number_of_neuron[a],
                                        kernel_initializer = kernel_initializers[b], activation = activation,
                                        kernel_regularizer = kernel_regularizer,
                                        activity_regularizer = activity_regularizer))
                            regressor.add(Dense(units = 1, kernel_initializer = kernel_initializers[b]))
                            regressor.compile(optimizer = optimizers[c] , loss = losses[d] , metrics=[metric_function])
                            #plot_model(regressor, to_file='model'+str(run_counter)+'.png')
                            callbacks = [EarlyStopping(monitor = early_stopping_monitoring, patience = early_stopping_patience)]
                            history = regressor.fit(X_train, y_train, 
                                                    batch_size = batch_sizes[e], epochs = 500, 
                                                    callbacks=callbacks, 
                                                    validation_data=(X_val, y_val), verbose = 1)
                            y_pred = regressor.predict(X_test)
                            mean_absolute_error = mean_absolute_error_eval(y_pred, y_test)
                            
                            #Saving the results
                            param_list[run_counter, 0] = number_of_neuron[a]
                            param_list[run_counter, 1] = kernel_initializers[b]
                            param_list[run_counter, 2] = optimizers[c]
                            param_list[run_counter, 3] = losses[d]
                            param_list[run_counter, 4] = batch_sizes[e]
                            param_list[run_counter, 5] = hidden_layer_counter + 1
                            param_list[run_counter, 6] = mean_absolute_error
                            #pickle.dump(regressor, open('Gewichte Durchlauf'+str(run_counter)+'.sav', 'wb'))
                            print('Durchlauf', run_counter + 1, 'von', len(param_list))
                            elapsed_time = time.time() - one_loop_time
                            print('Zeit für letzten loop:', elapsed_time)
                            
    complete_run_time = time.time() - complete_time
    print('time for grid search:', complete_run_time)                        
    return param_list

In [ ]:
kernel_initializer = ['he_uniform']
number_of_neurons = [25, 30, 35, 40, 45]
number_of_hidden_layers = [5, 6]
loss = [huber_loss, 'logcosh', 'mae', 'mape', 'mse']
param_list = grid_search(kernel_initializer = kernel_initializer, number_of_hidden_layers = number_of_hidden_layers, loss = loss, number_of_neurons = number_of_neurons)


In [ ]:
columns = ['number of neurons', 'kernel initializer', 'optimizer', 'loss function',
           'batch sizes', 'number of hidden layers', 'R² failure angle', 'MAE failure angle']
grid_search_list = pd.DataFrame(param_list, columns = columns)
grid_search_list = grid_search_list.sort_values(by='MAE failure angle')
grid_search_list

In [ ]:
kernel_initializer = ['he_uniform']
number_of_neurons = [45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 110, 120, 130, 140, 150]
number_of_hidden_layers = [6]
param_list = grid_search(kernel_initializer = kernel_initializer, number_of_hidden_layers = number_of_hidden_layers, number_of_neurons = number_of_neurons)


In [ ]:
columns = ['number of neurons', 'kernel initializer', 'optimizer', 'loss function',
           'batch sizes', 'number of hidden layers', 'R² failure angle', 'MAE failure angle']
grid_search_list = pd.DataFrame(param_list, columns = columns)
grid_search_list = grid_search_list.sort_values(by='MAE failure angle')
grid_search_list

In [ ]:
kernel_initializer = ['he_uniform']
number_of_neurons = [50, 51, 52, 53, 54, 55]
number_of_hidden_layers = [6]
param_list = grid_search(kernel_initializer = kernel_initializer, number_of_hidden_layers = number_of_hidden_layers, number_of_neurons = number_of_neurons)


In [ ]:
columns = ['number of neurons', 'kernel initializer', 'optimizer', 'loss function',
           'batch sizes', 'number of hidden layers', 'R² failure angle', 'MAE failure angle']
grid_search_list = pd.DataFrame(param_list, columns = columns)
grid_search_list = grid_search_list.sort_values(by='MAE failure angle')
grid_search_list